In [1]:
import tensorflow as tf

In [2]:
# 5.1 와인 데이터셋 불러오기
import pandas as pd
red = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
white = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep=';')
print(red.head())
print(white.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5 

In [4]:
# 5.2 와인 데이터셋 합치기
red['type'] = 0
white['type'] = 1

wine = pd.concat([red, white])

In [5]:
wine["type"].value_counts()

1    4898
0    1599
Name: type, dtype: int64

In [6]:
wine_norm = (wine - wine.min()) / (wine.max()-wine.min())
wine_norm.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,0.297521,0.413333,0.000000,0.019939,0.111296,0.034722,0.064516,0.206092,0.612403,0.191011,0.202899,0.333333,0.0
1,0.330579,0.533333,0.000000,0.030675,0.147841,0.083333,0.140553,0.186813,0.372093,0.258427,0.260870,0.333333,0.0
2,0.330579,0.453333,0.024096,0.026074,0.137874,0.048611,0.110599,0.190669,0.418605,0.241573,0.260870,0.333333,0.0
3,0.611570,0.133333,0.337349,0.019939,0.109635,0.055556,0.124424,0.209948,0.341085,0.202247,0.260870,0.500000,0.0
4,0.297521,0.413333,0.000000,0.019939,0.111296,0.034722,0.064516,0.206092,0.612403,0.191011,0.202899,0.333333,0.0


In [8]:
import numpy as np
wine_shuffle = wine_norm.sample(frac=1)
wine_np = wine_shuffle.to_numpy()

In [12]:
train_idx = int(len(wine_np)*0.8)
train_X, train_y = wine_np[:train_idx, :-1], wine_np[:train_idx,-1]
test_X, test_y = wine_np[train_idx:, :-1], wine_np[train_idx:,-1]
print(train_X[0])
print(train_y[0])
print(test_X[0])
print(test_y[0])

[0.39669421 0.12666667 0.27710843 0.08435583 0.03820598 0.04166667
 0.08064516 0.11355311 0.13178295 0.06741573 0.42028986 0.33333333]
1.0
[0.34710744 0.09333333 0.25301205 0.21472393 0.05813953 0.15277778
 0.36175115 0.25428957 0.37984496 0.26404494 0.08695652 0.66666667]
1.0


In [14]:
# 정답지를 one-hot-incoding을 해줘야 한다 (출력의 갯수를 맞추기 위해)
train_y = tf.keras.utils.to_categorical(train_y, num_classes=2)
train_y[0]

array([0., 1.], dtype=float32)

In [15]:
test_y = tf.keras.utils.to_categorical(test_y, num_classes=2)
test_y[0]

array([0., 1.], dtype=float32)

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=48, activation="relu",input_shape=(12,)),
    tf.keras.layers.Dense(units=24, activation="relu"),
    tf.keras.layers.Dense(units=12, activation='relu'),
    tf.keras.layers.Dense(units=2, activation="softmax")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.07),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 48)                624       
_________________________________________________________________
dense_9 (Dense)              (None, 24)                1176      
_________________________________________________________________
dense_10 (Dense)             (None, 12)                300       
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 26        
Total params: 2,126
Trainable params: 2,126
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.fit(train_X, train_y, epochs=25, batch_size=32, validation_split=0.25)

Epoch 1/25
122/122 [==============================] - 0s 2ms/step - loss: 0.1497 - accuracy: 0.9494 - val_loss: 0.0991 - val_accuracy: 0.9715
Epoch 2/25
122/122 [==============================] - 0s 1ms/step - loss: 0.0519 - accuracy: 0.9851 - val_loss: 0.0479 - val_accuracy: 0.9900
Epoch 3/25
122/122 [==============================] - 0s 1ms/step - loss: 0.0625 - accuracy: 0.9823 - val_loss: 0.0420 - val_accuracy: 0.9938
Epoch 4/25
122/122 [==============================] - 0s 1ms/step - loss: 0.0515 - accuracy: 0.9854 - val_loss: 0.0416 - val_accuracy: 0.9915
Epoch 5/25
122/122 [==============================] - 0s 1ms/step - loss: 0.0469 - accuracy: 0.9869 - val_loss: 0.0285 - val_accuracy: 0.9923
Epoch 6/25
122/122 [==============================] - 0s 1ms/step - loss: 0.0457 - accuracy: 0.9897 - val_loss: 0.0344 - val_accuracy: 0.9931
Epoch 7/25
122/122 [==============================] - 0s 1ms/step - loss: 0.0433 - accuracy: 0.9872 - val_loss: 0.0494 - val_accuracy: 0.9846
Epoch 